In [225]:
!pip install nltk

  Using cached https://files.pythonhosted.org/packages/84/cb/e7792d2c52f61a686ce0fabe79b3674c41238d07cd35c79b4062e9a807f6/regex-2021.3.17.tar.gz
  Using cached https://files.pythonhosted.org/packages/f8/3e/2730d0effc282960dbff3cf91599ad0d8f3faedc8e75720fdf224b31ab24/tqdm-4.59.0-py2.py3-none-any.whl
  Running setup.py bdist_wheel for regex: started
  Running setup.py bdist_wheel for regex: finished with status 'error'
  Complete output from command C:\Users\User\anaconda3\envs\tensorflow-gpu\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-rou9jhsm\\regex\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\User\AppData\Local\Temp\pip-wheel-w73y6c5j --python-tag cp35:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.5
  creating build\lib.win-amd64-3.5\regex
  cop

  Failed building wheel for regex
Command "C:\Users\User\anaconda3\envs\tensorflow-gpu\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-rou9jhsm\\regex\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\User\AppData\Local\Temp\pip-record-i52chhq6\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\User\AppData\Local\Temp\pip-install-rou9jhsm\regex\
You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [191]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [223]:
data = pd.read_csv('data_2021_spring/train.csv')
# Keeping only the neccessary columns
df = data[['text','stars']]

In [227]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
print(X[0])
X = pad_sequences(X, maxlen = 800)

[87, 4, 23, 5, 1620, 132, 146, 29, 6, 32, 2, 137, 5, 32, 355, 9, 322]


['nice to have a diner still around food was good and comforting definitely a good spot for breakfast'
 'tried this a while back got the fried chicken sandwich and it was meh it was pretty small and lacked the spices and flavors i was expecting in a chicken sandwich its nothing special in my opinion mary browns and kfcs chicken sandwiches are better\r\n\r\nit took super long to get 1 sandwich there was no one even in the store and it was about 7pm took a good 1520 minutes for the sandwich no combo the cashier was on her phone the whole time too not really updating me about my order'
 'i expected more pork selections on menu food very good beer selection good pork belly app one of the best ive had schnitzel also very good will be back oddly not busy for a friday night'
 ...
 'i first tried this place a while ago and ever since then i havent been disappointed \r\n\r\nthe place is small so its not great for big groups 6 plus just a heads up\r\n\r\nthe food there tastes like its prepped ri

In [229]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 800, 128)          256000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 800, 128)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 985       
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [230]:
Y = pd.get_dummies(data['stars']).values
print(X.shape,Y.shape)

(10000, 800) (10000, 5)


In [231]:
data2 = pd.read_csv('data_2021_spring/valid.csv')
# Keeping only the neccessary columns
data2 = data2[['text','stars']]

In [232]:
data2.head()

,text,stars
0,Small coffee shop on the corner. Good coffee. ...,5
1,Food was okay. Eating off styrofoam with plast...,3
2,Not at all impressed with Jet...maybe its beca...,2
3,"We got a 14"" pepperoni pizza I'd say it feeds ...",4
4,Unfortunately this location has terrible custo...,1


In [233]:
data2['text'] = data2['text'].apply(lambda x: x.lower())
data2['text'] = data2['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
max_fatures = 2000
# tokenizer.fit_on_texts(data2['text'].values)
X_valid = tokenizer.texts_to_sequences(data2['text'].values)
X_valid = pad_sequences(X_valid, maxlen = 800)

In [234]:
Y_train = pd.get_dummies(data['stars']).values
Y_valid = pd.get_dummies(data2['stars']).values

In [235]:
X_valid

array([[   0,    0,    0, ...,  313,  136,  109],
       [   0,    0,    0, ...,  370,  538,  160],
       [   0,    0,    0, ...,   15,   62,    4],
       ...,
       [   0,    0,    0, ...,  924,  104,  560],
       [   0,    0,    0, ..., 1146, 1920, 1465],
       [   0,    0,    0, ...,    2,   32,   29]])

In [236]:
Y_valid.shape

(2000, 5)

In [237]:
X_train = X
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)

(10000, 800) (10000, 5)
(2000, 800) (2000, 5)


In [ ]:
batch_size = 100
model.fit(X_train, Y_train, epochs = 12, batch_size=batch_size, verbose = 1, validation_data = (X_valid, Y_valid))

Epoch 1/12
100/100 [==============================] - 89s 892ms/step - accuracy: 0.3734 - loss: 1.4467 - val_accuracy: 0.5085 - val_loss: 1.1816
Epoch 2/12
100/100 [==============================] - 90s 897ms/step - accuracy: 0.5356 - loss: 1.0872 - val_accuracy: 0.5620 - val_loss: 1.0360
Epoch 3/12
100/100 [==============================] - 88s 882ms/step - accuracy: 0.5913 - loss: 0.9616 - val_accuracy: 0.5650 - val_loss: 1.0459
Epoch 4/12
100/100 [==============================] - 89s 889ms/step - accuracy: 0.6190 - loss: 0.9087 - val_accuracy: 0.5705 - val_loss: 1.0099
Epoch 5/12
100/100 [==============================] - 89s 891ms/step - accuracy: 0.6303 - loss: 0.8835 - val_accuracy: 0.5645 - val_loss: 1.0503
Epoch 6/12
100/100 [==============================] - 90s 896ms/step - accuracy: 0.6522 - loss: 0.8360 - val_accuracy: 0.5795 - val_loss: 1.0397
Epoch 7/12
100/100 [==============================] - 87s 874ms/step - accuracy: 0.6688 - loss: 0.8025 - val_accuracy: 0.5735 - va